<a href="https://colab.research.google.com/github/Ephrem758/Deep-learning/blob/main/Lab_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch

In [3]:
# Define a custom Dense layer
class MyDenseLayer:

    def __init__(self, input_dim, output_dim):
        # Initialize weights and bias randomly
        self.weights = torch.rand((input_dim, output_dim))
        self.bias = torch.rand(output_dim)

    def forward(self, inputs):
        # Perform forward pass through the layer
        self.output = torch.matmul(inputs, self.weights) + self.bias

In [4]:
# Define a custom Sigmoid activation function
class MySigmoidActivation:
    def forward(self, input):
        # Apply sigmoid activation function
        self.output = 1 / (1 + torch.exp(-input))
        return self.output

In [5]:
# Define a custom ReLU activation function
class MyReLUActivation:
    def forward(self, inputs):
        # Apply ReLU activation function
        self.output = torch.max(inputs, torch.tensor(0))
        return self.output

In [6]:
# Define a custom Categorical Cross-Entropy Loss function
class MyCategoricalLoss:
    def forward(self, y_pred, y_true):
        # Calculate categorical cross-entropy loss
        if y_pred.shape !=  y_true.shape:
            y_true -= 1
            one_hot_notation = torch.zeros(y_pred.shape)
            one_hot_notation[range(len(y_pred)), y_true] = 1
        else:
            one_hot_notation = y_pred
        loss = -torch.sum(one_hot_notation * torch.log(y_pred)) / len(y_true)
        self.output = loss
        return loss

In [7]:
# Define a custom Softmax activation function
class MySoftmaxActivation:
    def forward(self, inputs):
        # Apply softmax activation function
        pow = torch.exp(inputs - torch.max(inputs, axis=1, keepdims=True)[0])
        summ = torch.sum(pow, axis=1, keepdims=True)
        ans = pow / summ
        self.output = ans
        return ans

In [8]:
# Define a custom Classification Model
class MyClassificationModel:
    def __init__(self):
        # Initialize layers and activations
        self.layer1 = MyDenseLayer(2, 2)
        self.layer1_activation = MyReLUActivation()
        self.output_layer = MyDenseLayer(2, 2)
        self.output_layer_activation = MySigmoidActivation()
        self.error = float('inf')

    def forward_propagation(self, inputs):
        # Perform forward propagation
        self.input = inputs
        self.layer1.forward(inputs)
        self.layer1_activation.forward(self.layer1.output)
        self.output_layer.forward(self.layer1_activation.output)
        self.output_layer_activation.forward(self.output_layer.output)

    def calculate_error(self, y_true):
        # Calculate error using Mean Squared Error
        if self.output_layer_activation.output.shape != y_true.shape:
            one_hot_notation = torch.zeros(self.output_layer_activation.output.shape)
            one_hot_notation[y_true] = 1
        else:
            one_hot_notation = y_true
        self.y_true = one_hot_notation
        self.error = (self.output_layer_activation.output - one_hot_notation)
        self.mse = torch.mean(self.error)

    def backward_propagation(self, learning_rate):
        # Perform backward propagation
        dloss_by_dsig = self.error
        dsig_by_layer2 = (torch.tensor([1]) - self.output_layer_activation.output) * self.output_layer_activation.output

        # Layer 2 backpropagation
        back2 = dloss_by_dsig * dsig_by_layer2
        layer2_grad = torch.cat((self.layer1_activation.output.unsqueeze(dim=0),self.layer1_activation.output.unsqueeze(dim=0)),dim=0).T * back2
        self.output_layer.weights -= torch.tensor([learning_rate]) * layer2_grad
        self.output_layer.bias -= torch.tensor([learning_rate]) * back2

        # Layer 1 backpropagation
        drelu_by_dlayer1 = self.layer1.output > 0
        back1 = drelu_by_dlayer1 * torch.sum(self.output_layer.weights * back2, dim=1, keepdims=True).squeeze()
        layer1_grad = torch.cat((self.input.unsqueeze(dim=0),self.input.unsqueeze(dim=0)),dim=0).T * back1
        self.layer1.weights -= torch.tensor([learning_rate]) * layer1_grad
        self.layer1.bias -= torch.tensor([learning_rate]) * back1

    def train(self, inputs, y_true, epochs=1000, learning_rate=0.01):
        # Train the model
        for epoch in range(epochs):
            for i in range(len(inputs)):
                self.forward_propagation(inputs[i])
                self.calculate_error(y_true[i])
                if self.mse < 0.1:
                    break
                self.backward_propagation(learning_rate)
        error = torch.tensor([0, 0], dtype=torch.float)
        for i in range(len(inputs)):
            self.forward_propagation(inputs[i])
            self.calculate_error(y_true[i])
            print("Output:", self.output_layer_activation.output, "Expected:", self.y_true)
            print("Squared Error:", self.error**2)
            print()
            error += self.error**2

        avg_error = error / len(inputs)
        print("Average Squared Error:", avg_error)
        print("Overall Mean Squared Error:", torch.mean(avg_error))

In [9]:
# Instantiate the model and train it
model = MyClassificationModel()
inputs = torch.tensor([[0.5, 1.5], [1.5, 2.5], [4.5, 5.5], [6.5, 8.5], [6.5, 7.5], [0.5, 3.5]], dtype=torch.float)
y_true = torch.tensor([1, 1, 0, 0, 0, 1])
model.train(inputs, y_true, epochs=20000)

Output: tensor([0.3114, 0.8883]) Expected: tensor([0., 1.])
Squared Error: tensor([0.0969, 0.0125])

Output: tensor([0.4187, 0.9692]) Expected: tensor([0., 1.])
Squared Error: tensor([0.1753, 0.0009])

Output: tensor([0.7445, 0.9995]) Expected: tensor([1., 0.])
Squared Error: tensor([0.0653, 0.9990])

Output: tensor([0.8759, 1.0000]) Expected: tensor([1., 0.])
Squared Error: tensor([0.0154, 1.0000])

Output: tensor([0.8809, 1.0000]) Expected: tensor([1., 0.])
Squared Error: tensor([0.0142, 0.9999])

Output: tensor([0.2915, 0.9674]) Expected: tensor([0., 1.])
Squared Error: tensor([0.0850, 0.0011])

Average Squared Error: tensor([0.0754, 0.5022])
Overall Mean Squared Error: tensor(0.2888)
